In [1]:
import random
import copy
corpus0 = {'1.html': {'2.html'}, '2.html': {'1.html', '3.html'}, '3.html': {'2.html', '4.html'}, '4.html': {'2.html'}}
corpus1 = {'bfs.html': {'search.html'}, 'dfs.html': {'bfs.html', 'search.html'}, 'games.html': {'minesweeper.html', 'tictactoe.html'}, 'minesweeper.html': {'games.html'}, 'minimax.html': {'search.html', 'games.html'}, 'search.html': {'minimax.html', 'bfs.html', 'dfs.html'}, 'tictactoe.html': {'minimax.html', 'games.html'}}
corpus2 = {'ai.html': {'inference.html', 'algorithms.html'}, 'algorithms.html': {'recursion.html', 'programming.html'}, 'c.html': {'programming.html'}, 'inference.html': {'ai.html'}, 'logic.html': {'inference.html'}, 'programming.html': {'c.html', 'python.html'}, 'python.html': {'ai.html', 'programming.html'}, 'recursion.html': set()}

In [2]:
def transition_model(corpus, curr_page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """

    if len(corpus[curr_page]):
        #if page has outgoing links
        prdict = {pages: (
                damping_factor/len(corpus[curr_page]) +(1-damping_factor)/len(corpus) 
                if pages in corpus[curr_page] 
                else (1-damping_factor)/len(corpus))
                for pages in corpus
                }
        #print(prdict)
        
    else:#no outgoing links
        prdict = {pages: 1/len(corpus)
                for pages in corpus
                }
        #print(prdict)

    """
    The return value of the function should be a Python dictionary with one key for each page in the corpus. Each key should be mapped to a value representing the probability that a random surfer would choose that page next. The values in this returned probability distribution should sum to 1.
    {page:pr}
    """
    return prdict
    raise NotImplementedError

In [31]:
abc = transition_model(corpus0, "2.html", 0.85)
sum = 0
for a in abc:
    sum += abc[a]
    print(sum,abc[a])
    print(sum==1)

0.4625 0.4625
False
0.5 0.037500000000000006
False
0.9625 0.4625
False
1.0 0.037500000000000006
True


In [108]:
for i in corpus0:
    print(len(corpus0[i]))

probs = {
        next_page: 0
            
        for next_page in corpus0["2.html"]
    }
print(probs)

1
2
2
1
{'3.html': 0, '1.html': 0}


In [2]:
corpusa = {'1.html': {'2.html'}, '2.html': set() , '3.html': {'2.html', '4.html'}, '4.html': {'2.html'}}

In [84]:
def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """

    #random starting page
    r = random.randint(0,len(corpus)-1)
    count=0
    for i in corpus:
        if count == r:
            page = i
            break
        count+=1
    spdict = {pages: 0
              for pages in corpus
             }
    spdict[page] += 1/n

    for j in range(1,n):
        prdict = transition_model(corpus, page, damping_factor)
        r2=random.random()
        #Choosing next page by probability
        for i in prdict:
            r2 = r2 - prdict[i]
            if r2 < 0:
                page = i
                spdict[page] += 1/n
                break
            
    return spdict
    raise NotImplementedError

In [105]:
print(sample_pagerank(corpusa,0.85,1000000))

{'1.html': 0.14458700000012392, '2.html': 0.504230999993662, '3.html': 0.144673000000124, '4.html': 0.20650900000018585}


In [64]:
def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    ACC_DIFF = 0.001
    corpuscopy = copy.deepcopy(corpus)
    for p in corpuscopy:
        if not len(corpus[p]): #no outgoing links
             corpuscopy[p] = {s for s in corpus}

    itrdict = {pages: 1/len(corpus)
              for pages in corpus
             }

    flag = 0
    while(flag < len(itrdict)):#when all converged
        flag = 0
        itrdict_old = copy.deepcopy(itrdict)
        for p in itrdict:
            #PR(p) = (1-damping_factor)/len(corpus) + damping_factor*sum(PRi)/numLinks(i)
            
            sum_df = 0

            for i in corpuscopy:        #for all pages in corpus,
                if p in corpuscopy[i]:  #look for pages linking to p
                    #print(i,p,"left links to right")
                    sum_df += itrdict_old[i]/len(corpuscopy[i])
            itrdict[p] =  (1-damping_factor)/len(corpuscopy) + damping_factor*sum_df

            if abs(itrdict[p] - itrdict_old[p]) < ACC_DIFF:
                flag+=1
                #print(flag)
                #print(itrdict[p],itrdict_old[p])
            #else:
                #print("noflag",itrdict[p],itrdict_old[p],p)
    #print("converged")
    return itrdict
    raise NotImplementedError


In [65]:
print(iterate_pagerank(corpusa,0.85))

2.html 1.html left links to right
1.html 2.html left links to right
2.html 2.html left links to right
3.html 2.html left links to right
4.html 2.html left links to right
2.html 3.html left links to right
2.html 4.html left links to right
3.html 4.html left links to right
2.html 1.html left links to right
1.html 2.html left links to right
2.html 2.html left links to right
3.html 2.html left links to right
4.html 2.html left links to right
2.html 3.html left links to right
2.html 4.html left links to right
3.html 4.html left links to right
2.html 1.html left links to right
1.html 2.html left links to right
2.html 2.html left links to right
3.html 2.html left links to right
4.html 2.html left links to right
2.html 3.html left links to right
2.html 4.html left links to right
3.html 4.html left links to right
2.html 1.html left links to right
1.html 2.html left links to right
2.html 2.html left links to right
3.html 2.html left links to right
4.html 2.html left links to right
2.html 3.html 

In [72]:
print(True and False)
x,y = (1,2)
print(x)

False
1
